<font size="+3"><center>
    CCATpHive Documentation
</center></font>

<center><b>James Burgoyne</b></center>
<center>jburgoyne@phas.ubc.ca</center>

# Introduction  <a name="introduction"></a>

CCATpHive is python software to mediate communication between the control server and the parallel operating RFSoC boards which control readout operations for the CCAT/FYST telescope. Development was begun in January 2022. The software's main goals are to (1) provide interface hooks (and a terminal and graphical interface) to control computer and board functionality, (2) to automate the process of controlling the readout.

# Environment <a name="environment"></a>

Numerous environments and software versions have been used in the development and testing including modern Mac and Linux OS, Python versions 3.4-3.8, and Redis 5.0.3.

The control computer has the following software versions:
...

The boards run the following software versions:
...

# File Structure <a name="file_structure"></a>

## File Descriptions:
- **alcove_commands/**: Modules containing the functions to perform board tasks.
    - **board_io.py**: Extends base_io.py on the boards.
    - **board_utilities.py**: Basic board utility tools.
    - **init.py**: Initialization script. Must be run to initialize the board.
    - **multi_chan.py**: Gateware functions (four channel).
    - **single_chan.py**: Deprecated. Gateware functions (single channel).
    - **test_functions.py**: Testing functions.
- **docs/**: Documentation and guides.
- **drones/**: The board runs four drones which each have a subdirectory (droneX, X={1,2,3,4}).
    - **droneX/**: Configuration and data files specific to drone X.
        - **cal_tones/**: Directory to hold calibration tone files.
        - **targ/**: Directory to hold target sweep files.
        - **vna/**: Directory to hold VNA sweep files.
        - **_cfg_droneX.py**: Drone X specific configuration options.
- **gui_assets/**: Assets that queen_gui.py uses.
- **logs/**: Directory that contains log files.
- **queen_commands/**: Commands which run on the control server instead of the boards.
    - **control_io.py**: Extends base_io.py on the control computer.
    - **test_functions**: Testing functions.
- **RFnetworks/**: Contains a subdirectory for each RF network this board has used. Each subdirectory contains configuration and data files specific to that RF network.
- **tmp/**: Temporary files are dumped here. On the control computer this is primarily board returns.
- **_cfg_board.py**: RFSoC board configuration options. Note that this needs to be manually created on each board from _cfg_board.bak.py.
- **_cfg_queen.py**: Control server configuration options. Note that this needs to be manually created from _cfg_queen.bak.py.
- **alcove_tui.py**: A terminal interface to alcove.py. This is used only to directly interact with alcove.py when locally on the board.
- **alcove.py**: Provides an API to the board functionality functions (commands).
- **drone.py**: Runs on each of the boards (4 instances) and listens for commands from the control server (via Redis). Upon receiving a command it asks alcove.py to execute it and publishes returns. Must be running to receive commands.
- **init_multi**: Board initialization script for four channels. Must be run after bootup.
- **init**: Deprecated. Board initialization script for single channel.
- **queen_gui.ipynb**: Graphical interface to queen.py.
- **queen_tui.py**: Terminal interface to queen.py.
- **queen.py**: Runs on the command server to publish commands (via Redis) to remote boards, and to listen for messages from the boards. Should be running at all times in listen mode to pick up board messages.
- **quickDataViewer.ipynb**: A simple Jupyter notebook to inspect data in tmp/ (which are payloads from the board functions).

# Redis Channel Structure <a name="channel_structure"></a>

- **all_boards**: Channel to send commands to all boards at once.
- **board_[bid]**: Drones will listen to all channels that begin with this.
- **board_[bid].[drid]_[cid]**: Each board has it's own command channels. A new channel is created every time a command is issued with a random cid generated string suffix. [bid] and [drid] are the board and drone identifiers respectively (contained in _cfg_board.py) and [cid] is the command identifier which is a unique id generated when the command is sent.
- **rets_***: Boards send returns on the channel they received the command on modified with the prefix 'rets_'.

**[bid]**: Board identifier (contained in _cfg_board.py).  
**[drid]**: Drone identifier (1-4) (contained in _cfg_drone.py).  
**[cid]**: Command identifier which is a unique id generated when the command is sent.  

# Basic Usage <a name="basic_usage"></a>

**Initialization**:

1. Redis server startup (on control server).  
1. Listening Queen startup (on control server).  
1. Run multi_init.py script on boards.
1. Drone startup on boards: 4 drones per board.

**Basic command flow**:

1. Commands sent to Queen (e.g. via queen_tui.py or queen_gui.py) are communicated to redis in appropriate channels.
1. Drones are subscribed to redis and execute commands they receive.
1. Drones send return payloads to redis.
1. Listening Queen receives return values and saves/logs them.

## Redis Server
A Redis server must be running for the control server and boards to communicate. The configration options (e.g. host, port, etc.) for the Redis server are in ```_cfg_board.py``` for the boards, and in ```_cfg_queen.py``` for the control server. The Redis server will be hosted on the control server.

**Start Redis server:**
```bash
redis-server &
```
or to use a custom redis configuration file (to specify IP address and port, for example):
```bash
redis-server /usr/local/etc/redis.conf
```

**Stop Redis server:**  
CTRL-c from the terminal that started the redis server (unless it was started in the background using, e.g. using an ampersand after the start command).  
From an alternate terminal:
```bash
redis-cli shutdown
```

<figure>
  <img src="redis_startup.png" alt="Redis startup in the terminal."/>
  <figcaption>Redis server startup from a terminal. Note that if the ampersand wasn't addended, another terminal would be needed to shutdown the server.</figcaption>
</figure>

## Drone

Each board runs 4 drones (drone.py) in parallel (with configuration and data separated into 4 subdirectories), each operating a separate 512 MHz bandwidth microwave network (divided into 1000 channels). Each drone operates in a 'listening' mode in order to implement commands received from the queen over redis. The drones must be running at all times, and should be started at board startup, and up-status monitored via a daemon.

To start a single instance of drone.py with drone identifier of drid=1 (in listening mode):
```
python drone.py 1
```

## Interfaces

Queen (queen.py) can be interacted with from different user-interfaces. Currently there exists a TUI and GUI. There are plans to build a module for the telescope control software to interface with CCATpHive.

## TUI
Terminal interface to queen.py.

### Help

To get **help** with queen_tui and to see a list of all possible commands:
```
python queen_tui.py -h
```

### Commands

#### Command Number

All commands have an associated command number which is given as the first required positional argument. For example, to send the command with command number 1 (to all drones):  
```
python queen_tui.py 1
```

#### Command Recipient

A command can be for all drones, a specific board, a specific drone, or queen itself. If no recipient for a command is specified, it is assumed to be for all drones on all boards. 

To specify a command for a specific board or drone, use the second positional argument. The format is ```[bid]``` or ```[bid].[drid]```. For example, to send the command 1 to drone 3 on board 2:  
```
python queen_tui.py 1 2.3
```

To specify a command is for queen itself use the ```-q``` or ```--queen``` argument.

#### Command Returns

When a command is sent to a single drone, queen will output the return data. The form of the output data depends on it's type:
- **string**: The string is printed to the terminal. 
- **Numpy array**: The data is saved to a _.npy_ file in _tmp/_. This can be loaded using ```np.load()```.
- **Other types**: The data is written to an extensionless file in _tmp/_. This can be loaded using ```pickle.loads()```.

When a command is sent to multiple drones, the queen script used to send the command doesn't attempt to intercept the return. Instead, if an instance of queen is operating in listen mode, it will intercept all returns and save them to the _tmp/_ folder, as per the above scheme.

## GUI
The graphical interface to queen.py currently has the following features:
- Start/stop queen listening mode.
- Start/stop and capture a timestream from a select KID, and display in power or phase, with an adjustable range (packets/time).
- Send alcove commands with arguments to select boards or all boards.
- Display console output.

<figure>
  <img src="gui.png" alt="queen_gui.py"/>
  <figcaption>The graphical user interface (GUI) queen_gui.py for interfacing with queen.py.</figcaption>
</figure>

# Queen Commands <a name="queen_commands"></a>

## Listen Mode

An instance of queen (queen.py) should be running in 'listen' mode at all times to process board messages. 
To start a single instance of queen.py in **listen mode**:
```
python queen_tui.py 2 -q
```

# Board Commands <a name="board_commands"></a>
Current active commands are:
- 15:test.testFunction,
- 40:multi_chan.setNCLO,
- 41:multi_chan.writeTestTone,
- 42:multi_chan.writeVnaComb,
- 43:multi_chan.writeTargComb,
- 44:multi_chan.getSnapData,
- 45:multi_chan.vnaSweep,
- 46:multi_chan.findResonators,
- 47:multi_chan.targetSweep,
- 48:multi_chan.fullLoop,
- 49:multi_chan.loChop,
- 50:multi_chan.findCalTones,
- 51:multi_chan.updateTargComb,
- 52:multi_chan.setFineNCLO

# Changelog

## June 2022

**Update**: As of June 2022, a basic working version was complete including the ability to send commands from the control server to the boards via Redis and receive a return string, logging, basic configuration, and module integration to allow for board functionality. 

- Initial creation.

## August 2022

**Update**: In July and August additional functionality was integrated in to provide command arguments, return payloads, separation of the board into 4 drones (each controlling an RF network), key/value pairs, and a proof of concept GUI. 

## September 2022

**Update**: As of September 1st, the single channel module (single_chan) has rudimentary versions of the following functions: writeVnaComb, writeTestTone, getAdcData, getSnapData, vnaSweep, findResonators.

## December 2022

**Update**: multi_chan was added to control all 4 RF networks.

## May 2023

**Update**: The big change recently has been the addition of the GUI written in PyQt.The GUI has also incorporated a new and important feature for testing: Live time stream viewing and capturing. There is a known bug in (only?) the GUI which causes segmentation fault 11 frequently which is proving challenging to track down.

- Added Changelog.
- Removed date in title cell.
- Changed references to CCAT-Prime to CCAT/FYST.
- Rewrote the Introduction main goals.
- Updated the Environment section.
- Update the File Structure and Channel Structure sections.
- Update Basic Usage::Initialization to switch init.py -> multi_init.py and removed some wording.
- Updated Basic Usage::Basic command flow to add queen_gui.py to possible interfaces in step 1.
- Updated Basic Usage::Redis Server to change that queen Redis config options are in _cfg_queen.py instead of queen.py.
- Created a generic Interfaces section and update TUI section title from 'Queen (TUI)' to 'TUI'.
- Added a GUI section with a screenshot and feature list.
- Modified Queen Commands::Listen Mode command example to use the correct command number (21 -> 2).
- Filled in the current active commands in Board Commands section.